In [3]:
import numpy as np
import pandas as pd
import glob
import copy
import time
import re
import scipy.stats as stats

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_bdf, concatenate_raws
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch

import collections

from joblib import Parallel, delayed

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)

from mne_features.feature_extraction import extract_features

import sys

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
#==================================================================
# General Variables
#==================================================================
# Triggers
# 1 - New Spheres.
# 2 - Red Spheres.
# 4 - Start Moving.
# 8 - Stop Moving.
# 16 - TBD (~4 of them... and 4 blocks.)
# 32 - TBD (juste juste avant le 1, so I guess fin de trial, I have 88 of them... 4 trials)
trigger = 2

conds = ['1', '2', '3']
sides = ['right', 'left', 'classicmod']
perfs = ['good', 'bad']

# Making Custom Triggers and Event ID
# ------------------------------------
custom_trigger = dict()
custom_trigger['1'] = 100
custom_trigger['2'] = 200
custom_trigger['3'] = 300
custom_trigger['right'] = 10
custom_trigger['left'] = 20
custom_trigger['classicmod'] = 30
custom_trigger['good'] = 1
custom_trigger['bad'] = 2

# Making Event ID Dict to make it easy and readable to access (the desired) epochs.
event_id = dict()
for cond in conds:
    for side in sides:
        for perf in perfs:
            event_id['{}/{}/{}'.format(cond, side, perf)] = custom_trigger[cond] + custom_trigger[side] + custom_trigger[perf]
# ------------------------------------

# Channels & Clusters of Channels
# ------------------------------------
channels = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 
       'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 
       'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 
       'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 
       'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

clusters_chs = dict()
clusters_chs['Frontal'] = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'Fpz', 
                       'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8']
clusters_chs['Central']  = ['C1', 'C3', 'C5', 'C2', 'C4', 'C6', 'Cz', 'CP5', 'CP3', 'CP1', 
                        'CPz', 'CP6', 'CP4', 'CP2', 'FC5', 'FC3', 'FC1', 'FC6', 'FC4', 'FC2']
clusters_chs['Temporal'] = ['FT7', 'T7', 'TP7', 'FT8', 'T8', 'TP8']
clusters_chs['Parietal'] = ['TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 
                        'POz', 'Pz', 'CPz', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4']
clusters_chs['Occipital'] = ['O1', 'Oz', 'POz','O2']
clusters_chs['All'] = channels

clusters_names = {'F':'Frontal', 'P':'Parietal', 'C':'Central', 'O':'Occipital', 'T':'Temporal', 'All':'All'}

chan_right_all = ['O2', 'PO4', 'PO8', 'P2', 'P4', 'P6', 'P8', 'P10', 'C2', 'C4', 'C6', 'CP2', 'CP4', 'CP6', 'T8', 'TP8', 'FT8', 'FC2', 'FC4', 'FC6', 'F2', 'F4', 'F6', 'F8', 'AF4', 'AF8', 'Fp2']
chan_left_all =  ['O1', 'PO3', 'PO7', 'P1', 'P3', 'P5', 'P7', 'P9',  'C1', 'C3', 'C5', 'CP1', 'CP3', 'CP5', 'T7', 'TP7', 'FT7', 'FC1', 'FC3', 'FC5', 'F1', 'F3', 'F5', 'F7', 'AF3', 'AF7', 'Fp1']
chan_midline_all = ['AFz', 'Fz', 'FCz', 'Cz', 'Fpz', 'CPz', 'Pz', 'POz', 'Oz', 'Iz']
chan_others = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']

# ML Channels
ml_chan_left_all = [ch for ch in chan_left_all if ch in clusters_chs['Frontal']]
ml_chan_right_all = [ch for ch in chan_right_all if ch in clusters_chs['Frontal']]
ml_ch_pairs = []
for l,r in zip(ml_chan_left_all, ml_chan_right_all):
    ml_ch_pairs.append(f'{l}-{r}')
        
# ------------------------------------

# 3D-MOT Phases Information
phases = ['ID', 'Tracking', 'Recall']
stimx = {'ID':0, 'Tracking':2, 'Recall':11}
time_windows = {'ID':(-.2,2), 'Tracking':(1.5, 11), 'Recall':(10.5, 18)}

# Epoch Information
epoch_length = 20
epoch_tmin = -3
baseline_corr = (-1, -0.2)

# Folders & Paths
# directory_src = '/Users/nick/Documents/PhD/CLCT/NeuroTracker/Paper/src/'
# directory_data = '/Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Data/'
# directory_preproc = '/Users/nick/Documents/PhD/CLCT/NeuroTracker/Paper/data/'
directory_src = 'D:\\PhD\\_MOT_BCI_\\src\\'
directory_data = 'D:\\PhD\\_MOT_BCI_\\data - raw\\'
directory_preproc = 'D:\\PhD\\_MOT_BCI_\\data - preprocessed\\'

In [102]:
filepath = directory_data + 'F12F2.bdf' #'T25F1.bdf' #'T25F1.bdf'
raw = read_raw_bdf(filepath, preload=True)

raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3','EXG6','EXG7', 'EXG8'])
raw = raw.set_eeg_reference(ref_channels=['EXG4', 'EXG5'])
raw = raw.set_channel_types({'EXG4': 'misc', 'EXG5': 'misc'})

# Set Electrodes Locations
montage = mne.channels.make_standard_montage('biosemi64')
raw.set_montage(montage)

raw.resample(256)
raw = raw.filter(1, 40)

# Sampling Frequency
sfreq = raw.info['sfreq']

# Events 
events = find_events(raw)

# Load Behav Data.
df_behav = pd.read_csv(filepath[:filepath.rfind('.')] + '.csv', sep=',')

# Rework Events based on Behav Data. (to add Side and SetSize information)
new_events = []
nb_trials = 0
total_skipped = 0
for e in events:
    cur_cond = None
    cur_side = None
    cur_perf = None
    if e[2] == 2: # Find Start of Trial
        cur_cond = str(df_behav['NbTargets'].values[nb_trials]).lower()
        cur_side = str(df_behav['Mode'].values[nb_trials]).lower()
        cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
        if cur_cond is not None and cur_side is not None and cur_perf is not None:
            #temp_trigger = int(custom_trigger[cur_cond]) + int(custom_trigger[cur_side]) + int (custom_trigger[cur_perf])
            temp_event = [e[0], e[1], event_id['{}/{}/{}'.format(cur_cond, cur_side, cur_perf)]]
            new_events.append(temp_event) # Make a new list of events with new custom triggers.
        else:
            total_skipped = total_skipped + 1
            print('Skipping this event {}: No Cond or Side or Perf'.format(e))

        nb_trials = nb_trials + 1

        # Check Synch!
        #if (nb_trials + 1) % 10 == 0:
            #print('Check Synch!')

# Working with modified events from now on.
events = new_events

# Remove Events ID if not in file (e.g. some participants don't have bad trials with ss=1).
print('==================================================')
print('Removing Events ID not in EEG File...')
cur_event_id = copy.deepcopy(event_id)
newevents_in_file = set(np.array(events)[:,2])
keys_to_delete = []
for e in cur_event_id.keys():
    if event_id[e] not in newevents_in_file:
        print('Event: {} ({}), not present.'.format(e, event_id[e]))
        keys_to_delete.append(e)
for key in keys_to_delete:
    del cur_event_id[key]
print('==================================================') 

# Calculate average response time.
resp_time_in_ticks = []
for i, e in enumerate(events):
    if e[2] == 8:
        if events[i+1,2] == 32:
            resp_time_in_ticks.append(events[i+1,0] - events[i,0])
        elif events[i+1,2] == 16: #Synch?!
            if events[i+2,2] == 32:
                resp_time_in_ticks.append(events[i+2,0] - events[i,0])
            else:
                print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))
        else:
            print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))

mean_rt = np.mean(resp_time_in_ticks)
median_rt = np.median(resp_time_in_ticks)
std_rt = np.std(resp_time_in_ticks)
print('Response Time ~{:.2f}s (std:{:.2f})'.format(mean_rt/sfreq, std_rt/sfreq))

ml_event_id = dict()
ml_event_id['ml_left'] = 1
ml_event_id['ml_right'] = 0

new_merged_events = mne.merge_events(events, [121, 221, 321], ml_event_id['ml_left']) # Only Good for now.
new_merged_events = mne.merge_events(new_merged_events, [111, 211, 311], ml_event_id['ml_right'])  # Only Good for now.

Extracting EDF parameters from D:\PhD\_MOT_BCI_\data - raw\F12F2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5986303  =      0.000 ...  2923.000 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband

C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [95]:
# Epoching
epochs = Epochs(raw, new_merged_events, event_id=ml_event_id, tmin=11, tmax=12, picks='eeg', baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')

#epochs_bipolar = mne.set_bipolar_reference(epochs, anode=ml_chan_left_all, cathode=ml_chan_right_all, verbose=False)
#epochs_bipolar = epochs_bipolar.pick_channels(ml_ch_pairs)

epochs_bipolar = mne.set_bipolar_reference(epochs, anode=ml_chan_right_all, cathode=ml_chan_left_all, verbose=False)
epochs_bipolar = epochs_bipolar.pick_channels(ml_ch_pairs_inv)

#epochs_id = epochs_bipolar.copy().crop(tmin=0.15, tmax=1.15)

#epochs_bipolar = epochs

Not setting metadata
35 matching events found
Setting baseline interval to [11.0, 12.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 35 events and 257 original time points ...
0 bad epochs dropped


In [99]:
labels = epochs_bipolar.events[:, -1]
data = epochs_bipolar.get_data() * 1e6

print(data.shape)
print(labels)

(35, 8, 257)
[1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0]


In [100]:
import mne
import numpy as np
from mne.datasets import sample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from mne_features.feature_extraction import extract_features

In [101]:
###############################################################################
# Prepare for the classification task:

pipe = Pipeline([('scaler', StandardScaler()),
                 ('lr', LogisticRegression(random_state=42, solver='lbfgs'))])
y = labels

###############################################################################
# Classification using features (mean, peak-to-peak amplitude,
# standard deviation). See :ref:`api_documentation` for full list of supported
# features.

selected_funcs = {'mean', 'ptp_amp', 'std'}
X_new = extract_features(data, epochs.info['sfreq'], selected_funcs)
kf = KFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X_new, y, scoring='accuracy', cv=kf)

###############################################################################
# Print the cross-validation score:

print('Cross-validation accuracy score = %1.3f (+/- %1.5f)' % (np.mean(scores),
                                                               np.std(scores)))

Cross-validation accuracy score = 0.573 (+/- 0.05613)


# All Subjects Loop

In [107]:
#=============================================================================
# Main Loop - Loading Files 1 by 1 to get CDA per Cluster per Subject.
#=============================================================================
data_dict_all = dict()
scores_dict_all = dict()

sep = '\\'
#filepath = directory_data + 'W10F2.bdf'#'T25F1.bdf' #'F12F2.bdf' #'T25F1.bdf'
for filepath in glob.iglob(directory_data + '*.bdf'):
    filename = filepath[filepath.rfind(sep)+1:]
        
    # Skipping Exceptions with multiple EEG files. (they will be loaded with Block 1 (B1))
    if filename in ['M8M1_B3.bdf','M8M1_B4.bdf','M8M2_B3.bdf','M8M2_B4.bdf','T9F2_B3-B4.bdf','T9M1_B3-B4.bdf']:
        continue
    
    print('Processing: ' + filename + '...')

    # 1. -- Load File --
    if filename not in ['M8M1_B1-B2.bdf', 'M8M2_B1-B2.bdf', 'T9F2_B1-B2.bdf', 'T9M1_B1-B2.bdf']: # Check for exceptions with multiple EEG files.
        raw = read_raw_bdf(filepath, preload=True, verbose=False)
    else:
        if filename == 'M8M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True, verbose=False)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B3.bdf', preload=True, verbose=False)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B4.bdf', preload=True, verbose=False)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M1.bdf'
        if filename == 'M8M2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True, verbose=False)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B3.bdf', preload=True, verbose=False)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B4.bdf', preload=True, verbose=False)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M2.bdf'
        if filename == 'T9F2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True, verbose=False)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9F2_B3-B4.bdf', preload=True, verbose=False)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9F2.bdf'
        if filename == 'T9M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True, verbose=False)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9M1_B3-B4.bdf', preload=True, verbose=False)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9M1.bdf' 
            
    raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3','EXG6','EXG7', 'EXG8'])
    raw = raw.set_eeg_reference(ref_channels=['EXG4', 'EXG5'])
    raw = raw.set_channel_types({'EXG4': 'misc', 'EXG5': 'misc'})

    # Set Electrodes Locations
    montage = mne.channels.make_standard_montage('biosemi64')
    raw.set_montage(montage)

    # Downsampling
    raw.resample(256)
    # Filter
    #raw = raw.filter(4, 40)
  
    # Sampling Frequency
    sfreq = raw.info['sfreq']

    # Events 
    events = find_events(raw)

    # Load Behav Data.
    df_behav = pd.read_csv(filepath[:filepath.rfind('.')] + '.csv', sep=',')

    # Rework Events based on Behav Data. (to add Side and SetSize information)
    new_events = []
    nb_trials = 0
    total_skipped = 0
    for e in events:
        cur_cond = None
        cur_side = None
        cur_perf = None
        if e[2] == 2: # Find Start of Trial
            cur_cond = str(df_behav['NbTargets'].values[nb_trials]).lower()
            cur_side = str(df_behav['Mode'].values[nb_trials]).lower()
            cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
            if cur_cond is not None and cur_side is not None and cur_perf is not None:
                #temp_trigger = int(custom_trigger[cur_cond]) + int(custom_trigger[cur_side]) + int (custom_trigger[cur_perf])
                temp_event = [e[0], e[1], event_id['{}/{}/{}'.format(cur_cond, cur_side, cur_perf)]]
                new_events.append(temp_event) # Make a new list of events with new custom triggers.
            else:
                total_skipped = total_skipped + 1
                print('Skipping this event {}: No Cond or Side or Perf'.format(e))

            nb_trials = nb_trials + 1

            # Check Synch!
            #if (nb_trials + 1) % 10 == 0:
                #print('Check Synch!')

    # Working with modified events from now on.
    events = new_events

    # Remove Events ID if not in file (e.g. some participants don't have bad trials with ss=1).
    print('==================================================')
    print('Removing Events ID not in EEG File...')
    cur_event_id = copy.deepcopy(event_id)
    newevents_in_file = set(np.array(events)[:,2])
    keys_to_delete = []
    for e in cur_event_id.keys():
        if event_id[e] not in newevents_in_file:
            print('Event: {} ({}), not present.'.format(e, event_id[e]))
            keys_to_delete.append(e)
    for key in keys_to_delete:
        del cur_event_id[key]
    print('==================================================') 

    # Calculate average response time.
    resp_time_in_ticks = []
    for i, e in enumerate(events):
        if e[2] == 8:
            if events[i+1,2] == 32:
                resp_time_in_ticks.append(events[i+1,0] - events[i,0])
            elif events[i+1,2] == 16: #Synch?!
                if events[i+2,2] == 32:
                    resp_time_in_ticks.append(events[i+2,0] - events[i,0])
                else:
                    print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))
            else:
                print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))

    mean_rt = np.mean(resp_time_in_ticks)
    median_rt = np.median(resp_time_in_ticks)
    std_rt = np.std(resp_time_in_ticks)
    print('Response Time ~{:.2f}s (std:{:.2f})'.format(mean_rt/sfreq, std_rt/sfreq))

    ml_event_id = dict()
    ml_event_id['ml_left'] = 1
    ml_event_id['ml_right'] = 0

    new_merged_events = mne.merge_events(events, [121, 221, 321], ml_event_id['ml_left']) # Only Good for now.
    new_merged_events = mne.merge_events(new_merged_events, [111, 211, 311], ml_event_id['ml_right'])  # Only Good for now.
    
    # Epoching
    epochs = Epochs(raw, new_merged_events, event_id=ml_event_id, tmin=11, tmax=12.5, picks='eeg', baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')

    epochs_bipolar = mne.set_bipolar_reference(epochs, anode=ml_chan_left_all, cathode=ml_chan_right_all, verbose=False)
    epochs_bipolar = epochs_bipolar.pick_channels(ml_ch_pairs)

    labels = epochs_bipolar.events[:, -1]
    data = epochs_bipolar.get_data() * 1e6

    print(data.shape)
    print(labels)
    
    ###############################################################################
    # Prepare for the classification task:

    pipe = Pipeline([('scaler', StandardScaler()),
                     ('lr', LogisticRegression(random_state=42, solver='lbfgs'))])
    y = labels

    ###############################################################################
    # Classification using features (mean, peak-to-peak amplitude,
    # standard deviation). See :ref:`api_documentation` for full list of supported
    # features.

    selected_funcs = {'mean', 'ptp_amp', 'std'}
    X_new = extract_features(data, epochs.info['sfreq'], selected_funcs)
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipe, X_new, y, scoring='accuracy', cv=kf)

    ###############################################################################
    # Print the cross-validation score:

    print('Cross-validation accuracy score = %1.3f (+/- %1.5f)' % (np.mean(scores),
                                                                   np.std(scores)))
     
    data_dict_all[filename] = dict()
    data_dict_all[filename]['X'] = data
    data_dict_all[filename]['feat'] = X_new
    data_dict_all[filename]['y'] = labels
    
    scores_dict_all[filename] = scores
    
    print(f'Done with {filename}!')

Processing: F12F2.bdf...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Removing Events ID not in EEG File...
Event: 1/right/bad (112), not present.
Response Time ~nans (std:nan)
Not setting metadata
35 matching events found
Setting baseline interval to [11.0, 12.5] sec
Applying baseline correction (mode: mean)
0 project

C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Removing Events ID not in EEG File...
Event: 1/classicmod/bad (132), not present.
Response Time ~nans (std:nan)
Not setting metadata
43 matching events found
Setting baseline interval to [11.0, 12.5] sec
Applying baseline correction (mode: mean)
0 projection items activated


C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
Removing Events ID not in EEG File...
Event: 1/left/bad (122), not present.
Event: 1/classicmod/bad (132), not present.
Event: 2/left/bad (222), not present.
Response Time ~nans (std:nan)
Not setting metadata
48 matching events found
Setting baseline interval to [11.0, 12.5]

...

C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
516 events found
Event IDs: [ 1  2  4  8 16 32]
516 events found
Event IDs: [ 1  2  4  8 16 32]
516 events found
Event IDs: [ 1  2  4  8 16 32]
Removing Events ID not in EEG File...
Event: 1/left/bad (122), not present.
Response Time ~nans (std:nan)
Not setting metadata
41 matching events found
Setting baseline interval to [11.0, 12.5] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 41 events and 385 original time points ...
0 bad epochs dropped
(41, 8, 385)
[1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1
 1 1 0 1]
Cross-validation accuracy score = 0.756 (+/- 0.03054)
Done with M22F2.bdf!
Processing: M8M1_B1-B2.bdf...


C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Yannick\Miniconda3\envs\motml\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
645 events found
Event IDs: [    1     2     4     8    16    32 65536]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
645 events found
Event IDs: [    1     2     4     8    16    32 65536]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
645 events found
Event IDs: [    1     2     4     8    16    32 65536]


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PhD\\_MOT_BCI_\\data - raw\\M8M1_B1-B2.csv'